In [1]:
import torch
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import pickle
import time
from pathlib import Path

import onnx
import onnxruntime as ort
from onnx import helper, shape_inference

from torch2trt import TRTModule

In [8]:
model_path = Path('./TCN/mnist_pixel/models')
trt_model_path = Path('./TCN/mnist_pixel/models_trt')
data_path = Path('./TCN/mnist_pixel/data/mnist')
model_name = 'aug_k7l6'
batch_size = 1
in_channels = 1
n_classes = 10

_, test_loader = data_generator(data_path, batch_size)

# Torch Model

In [3]:
# Load Model

args = pickle.load(open(model_path / (model_name+'_args.pkl'), 'rb'))
print(args)
channel_sizes = [args.nhid] * args.levels

model = TCN(in_channels, n_classes, channel_sizes, kernel_size=args.ksize, trt=True)
model.load_state_dict(torch.load(model_path / (model_name+'.pt')), strict=False)
model.eval()

if torch.cuda.is_available():
    model.cuda()

Namespace(batch_size=64, clip=-1, cuda=True, dropout=0.05, epochs=50, ksize=7, levels=6, log_interval=100, lr=0.002, modelname='aug_k7l6', nhid=25, optim='Adam', permute=False, savedir=PosixPath('models'), savemodel=True, seed=-1, seq_augment=True)


In [4]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        output = model(data_queue)
        runtimes.append(time.time() - start)

torch_runtime = np.mean(runtimes)*1000
print(f'Average runtime per sample: {torch_runtime} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 3.1882090227944513 ms


# ONNX GPU Runtime

In [6]:
# Check Model
onnx_model = onnx.load(model_path / (model_name+'.onnx'))
onnx.checker.check_model(onnx_model)
inferred_model = shape_inference.infer_shapes(onnx_model)
onnx.checker.check_model(inferred_model)

# Load Session
onnx_path = model_path / (model_name+'.onnx')
ort_session = ort.InferenceSession(str(onnx_path))

In [7]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        ort_inputs = {ort_session.get_inputs()[0].name: data_queue.cpu().numpy()}
        ort_outs = ort_session.run(None, ort_inputs)
        runtimes.append(time.time() - start)

onnx_runtimes = np.mean(runtimes)*1000
print(f'Average runtime per sample: {onnx_runtimes} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 1.1331963295839271 ms


# TensorRT Models

## torch2trt using ONNX

In [11]:
# Load Model
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_onnx_amax.pt')))

<All keys matched successfully>

In [12]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        output = model_trt(data_queue)
        runtimes.append(time.time() - start)

tensorrt_runtimes = np.mean(runtimes)*1000
print(f'Average runtime per sample: {tensorrt_runtimes} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 0.8545281631605965 ms


## torch2trt using converters

In [13]:
# Load Model
model_trt_no_onnx = TRTModule()
model_trt_no_onnx.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_amax.pt')))

<All keys matched successfully>

In [30]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        output = model_trt_no_onnx(data_queue)
        runtimes.append(time.time() - start)

tensorrt_no_onnx_runtimes = np.mean(runtimes)*1000
print(f'Average runtime per sample: {tensorrt_no_onnx_runtimes} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 0.9404097892800156 ms


## torch2trt with ONNX - FP16

In [18]:
# Load Model
model_trt_fp16 = TRTModule()
model_trt_fp16.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_onnx_amax_fp16.pt')))

<All keys matched successfully>

In [19]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        output = model_trt_fp16(data_queue)
        runtimes.append(time.time() - start)

tensorrt_runtimes_fp16 = np.mean(runtimes)*1000
print(f'Average runtime per sample: {tensorrt_runtimes_fp16} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 0.8252052020053474 ms


## torch2trt with converters - FP16

In [20]:
# Load Model
model_trt_no_onnx_fp16 = TRTModule()
model_trt_no_onnx_fp16.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_amax_fp16.pt')))

<All keys matched successfully>

In [26]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time() 
        output = model_trt_no_onnx_fp16(data_queue)
        runtimes.append(time.time() - start)

tensorrt_no_onnx_runtimes = np.mean(runtimes)*1000
print(f'Average runtime per sample: {tensorrt_no_onnx_runtimes} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 0.8086569759310509 ms


In [29]:
num_samples = 10*784
runtimes = []

for i in range(num_samples):
    if i % 784 == 0: print(f'Running sample image {i//784}')
    x = torch.rand((1,1,28*28)).cuda()
    
    start = time.time()
    output = model_trt_no_onnx(x)
    runtimes.append(time.time() - start)

test_runtimes = np.mean(runtimes)*1000
print(f'Average runtime per sample: {test_runtimes} ms')

Running sample image 0
Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Average runtime per sample: 0.8949189769978426 ms


In [ ]:
print(f'ONNX Runtime is {torch_runtime/onnx_runtimes} times faster than PyTorch (gpu)')
print(f'TensorRT (using onnx) Runtime is {torch_runtime/tensorrt_runtimes} times faster than regular PyTorch')
print(f'TensorRT (no onnx) Runtime is {torch_runtime/tensorrt_no_onnx_runtimes} times faster than regular PyTorch')